In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl datasets torch
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 23.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 19.2 MB/s eta 0:00:00


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import accelerate
import peft
import bitsandbytes
import transformers
import trl
import datasets

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import TrainingArguments
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

import warnings
warnings.filterwarnings('ignore')

In [5]:
if torch.cuda.is_available():
    print(torch.cuda.device_count(), torch.cuda.get_device_name(0), torch.cuda.get_device_properties(0).total_memory / 1e9)

2 Tesla T4 15.828320256


In [6]:
dataset = load_dataset("nlpie/Llama2-MedTuned-Instructions")

README.md:   0%|          | 0.00/2.96k [00:00<?, ?B/s]

(…)-00000-of-00001-a8790d88efc2bc45.parquet:   0%|          | 0.00/91.1M [00:00<?, ?B/s]

(…)-00000-of-00001-b543c64b1786c03e.parquet:   0%|          | 0.00/6.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200252 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/70066 [00:00<?, ? examples/s]

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 200252
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 70066
    })
})

In [9]:
# Show the first 3 rows
for i in range(3):
    data = dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Instruction >>>", data['instruction'])
    print("Input       >>>", data['input'])
    print("Output      >>>", data['output'])
    print("\n-----------------------------\n")

Data Point 1:
Instruction >>> In your role as a medical professional, address the user's medical questions and concerns.
Input       >>> My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.
Output      >>> Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.

-----------

In [10]:
# selecting some data to train the model fast
dataset["train"] = dataset["train"].select(range(10000))
dataset["test"]  = dataset["validation"].select(range(1000))

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 70066
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'source'],
        num_rows: 1000
    })
})

In [12]:
# creating the prompt 
def create_prompt(sample):
    prompt = str(sample["instruction"])
    prompt += " " + str(sample["input"])
    
    single_turn_prompt = f"""Instruction: {prompt}<|end_of_turn|> \n\nAI Assistant: {sample["output"]}"""
    return [single_turn_prompt]

In [13]:
print(create_prompt(dataset['train'][0])[0])

Instruction: In your role as a medical professional, address the user's medical questions and concerns. My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.<|end_of_turn|> 

AI Assistant: Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.


In [14]:
print(create_prompt(dataset['train'][10])[0])

Instruction: In the clinical text, your objective is to identify relationships between medical problems, treatments, and tests. Medical problems are tagged as @problem$, medical tests as @test$, and treatments as @treatment$. Classify the relationship between two entities as one of the following:
Treatment improves medical problem (TrIP)
Treatment worsens medical problem (TrWP)
Treatment causes medical problem (TrCP)
Treatment is administered for medical problem (TrAP)
Treatment is not administered because of medical problem (TrNAP)
Test reveals medical problem (TeRP)
Test conducted to investigate medical problem (TeCP)
Medical problem indicates medical problem (PIP)
No Relations Digoxin 0.125 mg q.d. , @treatment$ 80 mg q.a.m. and 40 mg q.p.m. aspirin 1 q.d. , and @treatment$ three puffs b.i.d.<|end_of_turn|> 

AI Assistant: No Relations


In [15]:
# Quantization step
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype="float16",
                                bnb_4bit_use_double_quant=True)

In [16]:
# Loading the LLM
# https://huggingface.co/berkeley-nest/Starling-LM-7B-alpha

repository_hf = "berkeley-nest/Starling-LM-7B-alpha"


# Load the LLM applying quantization
llm_model = AutoModelForCausalLM.from_pretrained(
    repository_hf,
    quantization_config=bnb_config,
    device_map="auto",
    use_cache=False
    )

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

In [17]:
# Load the LLM tokenizer
tokenizer = AutoTokenizer.from_pretrained(repository_hf)

# Padding 
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

In [18]:
tokenizer.eos_token, tokenizer.eos_token_id

('<|end_of_turn|>', 32000)

In [19]:
def generate_response_before_fine_tuning(prompt, model):

    # Apply tokenizer
    encoded_input = tokenizer(prompt,
                              return_tensors="pt", # pytorch
                              add_special_tokens=True)

    # Input to tensor
    model_inputs = encoded_input.to("cuda")

    # Generate response
    generated_ids = model.generate(**model_inputs,
                                   max_new_tokens=1024,
                                   do_sample=True,
                                   pad_token_id=tokenizer.eos_token_id)

    # Decoding the response
    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0].replace(prompt, "")

In [20]:
# Example
prompt = """Instruction: Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:
Contradiction: If the two sentences contradict each other. Neutral: If the two sentences are unrelated to each other. Entailment: If one of the sentences logically entails the other. """
prompt += '''Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension. Sentence 2: the patient has orthostatic hypotension <|end_of_turn|>'''
prompt += "\n\nAI Assistant: "

In [21]:
print(generate_response_before_fine_tuning(prompt, llm_model))

<s> Instruction: Your goal is to determine the relationship between the two provided clinical sentences and classify them into one of the following categories:
Contradiction: If the two sentences contradict each other. Neutral: If the two sentences are unrelated to each other. Entailment: If one of the sentences logically entails the other. Sentence 1: For his hypotension, autonomic testing confirmed orthostatic hypotension. Sentence 2: the patient has orthostatic hypotension <|end_of_turn|> 

AI Assistant: 

Answer: Entailment

Explanation: Sentence 1 provides a clinical statement about a patient with hypotension that was confirmed by autonomic testing as orthostatic hypotension. Sentence 2 also states that a patient has orthostatic hypotension. The information provided in Sentence 1 logically entails the statement made in Sentence 2, as both discuss the same diagnosis of orthostatic hypotension. 

Therefore, the two sentences are in a relationship of entailment, where the information

In [22]:
torch.cuda.empty_cache()  # Clear the GPU memory cache
torch.cuda.init()  # Force GPU initialization

In [23]:
# Define LoRA parameters with PEFT
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

In [24]:
# Prepare the model for the fine-tuning step
llm_finetuned_model = prepare_model_for_kbit_training(llm_model)

# Concatenate the base model with the LoRA parameters
llm_finetuned_model = get_peft_model(llm_finetuned_model, peft_config)

# llm_finetuned_model.to(device)  # Move the model to the GPU explicitly

In [25]:
# # Model hyperparameters
# accelerator = accelerate.Accelerator()
# local_rank = accelerator.process_index
# local_rank

In [26]:
os.environ["WANDB_DISABLED"] = "true"

In [27]:
# Checking LoRA
print("\nChecking LoRA configurations:")
print(f"LoRA active modules: {llm_finetuned_model.peft_config['default'].target_modules}")
print(f"# trainable parameters: {sum(p.numel() for p in llm_finetuned_model.parameters() if p.requires_grad)}")

trainable_params = []
for name, param in llm_finetuned_model.named_parameters():
    if param.requires_grad:
        trainable_params.append(name)
        
print(f"# trainable parameters list: {len(trainable_params)}")
# for param in trainable_params:
#     print(param)


Checking LoRA configurations:
LoRA active modules: {'q_proj', 'v_proj', 'k_proj', 'o_proj', 'gate_proj', 'down_proj', 'up_proj'}
# trainable parameters: 20971520
# trainable parameters list: 448


In [28]:
training_arguments = TrainingArguments(
    output_dir="./model_finetuned",
    
    # Batch configuration and learning
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    
    # Optimizer and scheduler
    optim="adamw_torch",  # paged_adamw_32bit
    lr_scheduler_type="cosine",
    
    # Precision and device
    bf16=False,  # Unset bfloat16
    fp16=True if torch.cuda.is_available() else False,  # Set fp16 if GPU is available
    
    # Saving and logging strategies
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    logging_steps=1,
    logging_strategy="steps",
    
    # Epochs and evaluation
    num_train_epochs=1,
    max_steps=5, # 250,
    evaluation_strategy="steps",
    eval_steps=50,
    
    # Extra configurations
    # remove_unused_columns=False,  # must have on SFTTrainer
    disable_tqdm=False,
    log_level="info",
    report_to=[],
    
    # GPU configurations
    no_cuda=False if torch.cuda.is_available() else True,
    
    # Addition configurations
    warmup_steps=50,
    weight_decay=0.01,
    max_grad_norm=1.0,
)

# Supervised Fine-Tuning Trainer (SFTT) https://huggingface.co/docs/trl/sft_trainer
# Initialize trainer
trainer = SFTTrainer(
    model=llm_finetuned_model,
    peft_config=peft_config,
    tokenizer=tokenizer,
    formatting_func=create_prompt,
    args=training_arguments,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

You have loaded a model on multiple GPUs. `is_model_parallel` attribute will be force-set to `True` to avoid any unexpected behavior such as device placement mismatching.
max_steps is given, it will override any value given in num_train_epochs
Using auto half precision backend


In [29]:
print("Pre-training verifications:")
print(f"# trainable paramaters : {sum(p.numel() for p in llm_finetuned_model.parameters() if p.requires_grad)}")
print(f"model device           : {next(llm_finetuned_model.parameters()).device}")
print(f"model data type        : {next(llm_finetuned_model.parameters()).dtype}")

Pre-training verifications:
# trainable paramaters : 20971520
model device           : cuda:0
model data type        : torch.float32


In [30]:
# from transformers import logging

# logging.set_verbosity_info()  # Set verbosity to show detailed information

# print("Is GPU available?", torch.cuda.is_available())
# print("Using device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# print(f"Accelerate device: {accelerate.Accelerator().device}")
# print(f"Current device index: {torch.cuda.current_device()}")

# torch.cuda.empty_cache()  # Clear the GPU memory cache
# torch.cuda.init()  # Force GPU initialization

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:
%%time 
trainer.train()

***** Running training *****
  Num examples = 10
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 5
  Number of trainable parameters = 20,971,520


Step,Training Loss,Validation Loss


Saving model checkpoint to ./model_finetuned/checkpoint-5
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--berkeley-nest--Starling-LM-7B-alpha/snapshots/1dddf3b95bc1391f6307299eb1c162c194bde9bd/config.json
Model config MistralConfig {
  "_name_or_path": "openchat/openchat_3.5",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.0",
  "use_cache": true,
  "vocab_size": 32002
}

tokenizer config file saved in ./model_finetuned/checkpoint-5/tok

CPU times: user 43.9 s, sys: 32 s, total: 1min 15s
Wall time: 1min 16s


TrainOutput(global_step=5, training_loss=1.1271696209907531, metrics={'train_runtime': 75.7842, 'train_samples_per_second': 0.264, 'train_steps_per_second': 0.066, 'total_flos': 788702309646336.0, 'train_loss': 1.1271696209907531, 'epoch': 1.8})

In [32]:
import os
import shutil
# from huggingface_hub import HfApi
# from peft import PeftModel

def save_trained_model(trainer, output_dir="./model_finetuned", zip_name="finetuned_model.zip"):
    # (model, tokenizer, output_dir="./model_output", zip_name="trained_model.zip")
    """
    Save the model, the tokenizer and configurations in a zip file
    """

    trainer.save_model(output_dir)
    
    # os.makedirs(output_dir, exist_ok=True)
    
    # # 1. Saving the LoRA model
    # print("Saving LoRA adapters...")
    # model.save_pretrained(output_dir)
    
    # # 2. Saving the tokenizer
    # print("Saving the tokenizer...")
    # tokenizer.save_pretrained(output_dir)
    
    # 3. Crie o arquivo zip
    print("Zipping...")
    shutil.make_archive(zip_name.replace('.zip', ''), 'zip', output_dir)
    
    print(f"Model saved at: {zip_name}")
    
    # 4. File size
    zip_size = os.path.getsize(zip_name) / (1024 * 1024)  # Tamanho em MB
    print(f"Zip size: {zip_size:.2f} MB")
    
    # if os.path.exists('/kaggle/working'):
    #     output_path = '/kaggle/working/' + zip_name
    #     if os.path.exists(zip_name):
    #         shutil.move(zip_name, output_path)
    #         print(f"File moved to: {output_path}")

def load_model_with_memory_optimization(
    base_model_name,
    adapter_path,
    use_4bit=True,
    device_map="auto"
    ):
    """
    Load the model with memory optimization
    """
    # Clean CUDA memory
    torch.cuda.empty_cache()
    
    # CUDA memory optimization config
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    
    # 4-bits quantization
    if use_4bit:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
        )
    else:
        quantization_config = None
    
    try:
        # Load model
        print("Loading the base model...")
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            quantization_config=quantization_config,
            torch_dtype=torch.float16,
            device_map=device_map,
            offload_folder="offload",  # Allow offload to CPU
            offload_state_dict=True,   # Allow offload of state dict
            low_cpu_mem_usage=True,    # Optimize the CPU memory usage
        )
        
        # Load tokenizer
        print("Load tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(adapter_path)
        
        # Load LoRA adapters
        print("Applying LoRA adapters...")
        model = PeftModel.from_pretrained(
            base_model,
            adapter_path,
            device_map=device_map,
            is_trainable=False  # Define como False se for apenas para inferência
        )
        
        print("Model loaded with success!")
        return model, tokenizer
        
    except Exception as e:
        print(f"Error loading the model: {str(e)}")
        if torch.cuda.is_available():
            print(f"Available GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
            print(f"Used GPU: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
        raise e

# save_trained_model(
#     model=llm_finetuned_model,
#     tokenizer=tokenizer,
#     output_dir="./model_output",
#     zip_name="llm_finetuned_model.zip"
# )

In [33]:
# repository_hf = "berkeley-nest/Starling-LM-7B-alpha"

# model, tokenizer = load_trained_model(
#     base_model_name=repository_hf,  # ex: "mistralai/Mistral-7B-v0.1"
#     adapter_path="./model_output"
# )

In [34]:
# import shutil

# shutil.make_archive("trained_model.zip".replace('.zip', ''), 'zip', "./model_finetuned")

In [35]:
save_trained_model(trainer) # output_dir="./model_finetuned", zip_name="finetuned_model.zip"

Saving model checkpoint to ./model_finetuned
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--berkeley-nest--Starling-LM-7B-alpha/snapshots/1dddf3b95bc1391f6307299eb1c162c194bde9bd/config.json
Model config MistralConfig {
  "_name_or_path": "openchat/openchat_3.5",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 32000,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.0",
  "use_cache": true,
  "vocab_size": 32002
}

tokenizer config file saved in ./model_finetuned/tokenizer_config.json
Special

Zipping...
Model saved at: finetuned_model.zip
Zip size: 516.98 MB


In [36]:
fine_tuned_model = llm_finetuned_model.merge_and_unload()

In [37]:
def generate_response(prompt, model):

    encoded_input = tokenizer(prompt,
                              return_tensors="pt",
                              add_special_tokens=True)

    model_inputs = encoded_input.to("cuda")

    generated_ids = model.generate(**model_inputs,
                                   max_new_tokens=512,
                                   do_sample=True,
                                   use_cache=False,
                                   pad_token_id=tokenizer.eos_token_id)

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0]

In [7]:
%%time
prompt = "Instruction: In your role as a medical professional, address the user's medical questions and concerns. "
prompt += "I have a white tab under my tounge that is not only painful when i touch it but bleeds as well. not sure what it is, or why I got it. Can you give me any advise?"
prompt += " <|end_of_turn|> "
prompt += "\n\nAI Assistant: "
response = generate_response(prompt, fine_tuned_model)

from pprint import pprint
pprint(response)

In [1]:
## "deploy"
# reset kernel

!pip install -q accelerate peft bitsandbytes transformers trl datasets torch
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 21.0 MB/s eta 0:00:00


In [2]:
import accelerate
import peft
import bitsandbytes
import transformers
import trl
import datasets

import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import TrainingArguments
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer

import warnings
warnings.filterwarnings('ignore')

In [3]:
import os
import shutil
# from huggingface_hub import HfApi
# from peft import PeftModel

def load_model_with_memory_optimization(
    base_model_name,
    adapter_path,
    use_4bit=True,
    device_map="auto"
    ):
    """
    Load the model with memory optimization
    """
    # Clean CUDA memory
    torch.cuda.empty_cache()
    
    # CUDA memory optimization config
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
    
    # 4-bits quantization
    if use_4bit:
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
        )
    else:
        quantization_config = None
    
    try:
        # Load model
        print("Loading the base model...")
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            quantization_config=quantization_config,
            torch_dtype=torch.float16,
            device_map=device_map,
            offload_folder="offload",  # Allow offload to CPU
            offload_state_dict=True,   # Allow offload of state dict
            low_cpu_mem_usage=True,    # Optimize the CPU memory usage
        )
        
        # Load tokenizer
        print("Load tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(adapter_path)
        
        # Load LoRA adapters
        print("Applying LoRA adapters...")
        model = PeftModel.from_pretrained(
            base_model,
            adapter_path,
            device_map=device_map,
            is_trainable=False  # False if inference mode
        )
        
        print("Model loaded with success!")
        return model, tokenizer
        
    except Exception as e:
        print(f"Error loading the model: {str(e)}")
        if torch.cuda.is_available():
            print(f"Available GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
            print(f"Used GPU: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
        raise e

In [4]:
!ls

finetuned_model.zip  model_finetuned  state.db	trained_model.zip


In [ ]:
repository_hf = "berkeley-nest/Starling-LM-7B-alpha"

model, tokenizer = load_model_with_memory_optimization(
    base_model_name=repository_hf,  # ex: "mistralai/Mistral-7B-v0.1"
    adapter_path="./model_finetuned",
    use_4bit=False
)

Loading the base model...


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]